In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
import rhealpixdggs.dggs as dggs
from rhealpixdggs.dggs import RHEALPixDGGS, Cell, WGS84_003, WGS84_003_RADIANS
from rhealpixdggs.ellipsoids import (
    Ellipsoid,
    WGS84_ASPHERE_RADIANS,
    WGS84_ELLIPSOID,
    WGS84_ELLIPSOID_RADIANS,
)

In [6]:
from typing import Union, List
from shapely.geometry import mapping, Polygon, GeometryCollection
from shapely import affinity

# https://towardsdatascience.com/around-the-world-in-80-lines-crossing-the-antimeridian-with-python-and-shapely-c87c9b6e1513
# https://gist.github.com/PawaritL/ec7136c0b718ca65db6df1c33fd1bb11

def check_crossing(lon1: float, lon2: float, validate: bool = True):
    """
    Assuming a minimum travel distance between two provided longitude coordinates,
    checks if the 180th meridian (antimeridian) is crossed.
    """
    if validate and any(abs(x) > 180.0 for x in [lon1, lon2]):
        raise ValueError("longitudes must be in degrees [-180.0, 180.0]")   
    return abs(lon2 - lon1) > 180.0

def check_for_geom(geom):
    crossed = False
    p_init = geom.exterior.coords[0]

    for p in range(1, len(geom.exterior.coords)):
        px = geom.exterior.coords[p]
        # print(px)

        if check_crossing(p_init[0], px[0]):
            crossed = True
        p_init = px
    
    return crossed

In [7]:
def __lonlat_to_latlon(lonlat_array):
    latlon_array = []
    for vertex in lonlat_array:
        latlon_array.append((vertex[1],vertex[0]))
    return latlon_array

def __cell_to_geometry(cell):
    geom = None
    try:
        geom =  Polygon(cell.boundary(n=2,plane=False))
    except:
        print(f'internal rhealpix error with cell.boundary method for {str(cell)}')
    return geom

def create_rhpix_geometry(df):

    gdf = gpd.GeoDataFrame(df.copy())
    gdf['geometry'] = gdf['cell_id'].apply(__cell_to_geometry)
    gdf.crs = 'EPSG:4326'
    gdf['cell_id'] = gdf['cell_id'].apply(lambda x: str(x))

    return gdf


def get_rhpix_cells(res, extent=None):
    rdggs = WGS84_003
    if extent:
        nw = (extent[1], extent[2])
        se = (extent[3], extent[0])
        set_hex = list(flatten(rdggs.cells_from_region(res, nw, se, plane=False)))
    else:    
        set_hex = [x for x in rdggs.grid(res)]

    df = pd.DataFrame({"cell_id": set_hex})
    
    return df


def create_rhpix_geom_cells_global(resolutions, table, export_type, db_engine=''):
    """Create geometry for rhpix cells globally for given resolutions
        Parameters:
        db_engine (sqlalchemy.engine): sqlalchemy database engine
        resolutions(array): array of integer h3 resolution levels
        table(string): table name for postgres database
        export_type(string): where to export 'geojson' or 'postgres'
        Returns:
        none
    """
    rdggs = WGS84_003
    transformer = Transformer.from_crs("epsg:4326", 'proj=rhealpix')
    for res in resolutions:
        
        gdf = gpd.GeoDataFrame({'cell_id':[x for x in rdggs.grid(res)]})
        gdf['geometry'] = gdf['cell_id'].apply(lambda x: Polygon(x.boundary(n=10,plane=False)))
        gdf.crs = 'EPSG:4326'
        gdf['cell_id'] = gdf['cell_id'].apply(lambda x: str(x))
        gdf['area'] = gdf['geometry'].apply(lambda x: transform(transformer.transform, x).area)
        
        print('finish caclulating geometry {} {}'.format(res, time.asctime(time.localtime(time.time()))))
        
        if export_type == 'postgres':
            
            gdf.to_postgis(table + str(res), db_engine, if_exists='replace')
            print('finish import to db {} {}'.format(res, time.asctime(time.localtime(time.time()))))

            
        elif export_type == 'geojson':

            gdf.to_file("{}{}.geojson".format(table, res), driver='GeoJSON')
            print('finish import to geojson {} {}'.format(res, time.asctime(time.localtime(time.time()))))


In [8]:
gdf_rhpx_sm = create_rhpix_geometry( get_rhpix_cells(2, None) )

gdf_rhpx = create_rhpix_geometry( get_rhpix_cells(3, None) )

In [9]:
gdf_rhpx_sm['crossed'] = gdf_rhpx_sm['geometry'].apply(check_for_geom)
display(gdf_rhpx_sm['crossed'].value_counts())
gdf_rhpx_sm = gdf_rhpx_sm.loc[gdf_rhpx_sm['crossed'] == False]

gdf_rhpx['crossed'] = gdf_rhpx['geometry'].apply(check_for_geom)
display(gdf_rhpx['crossed'].value_counts())
gdf_rhpx = gdf_rhpx.loc[gdf_rhpx['crossed'] == False]

False    459
True      27
Name: crossed, dtype: int64

False    4293
True       81
Name: crossed, dtype: int64

In [10]:
test_est = pd.read_csv("rhpix_agg.csv")
test_est.head(2)

,cell_id,elevation,corine_landuse,temperature,precipitation,population,soil_class
0,N23331748,0.0,523,0.0,0.0,0.0,21.0
1,N23331771,0.0,523,0.0,0.0,0.0,21.0


In [11]:
test_est.size

258965

In [12]:
len(test_est.index)

36995

In [13]:
rdggs = WGS84_003

In [27]:
def rhpix_suid_to_string(suid):
    return "".join(map(lambda x: str(x), suid))


def rhpix_string_to_suid(s):
    b = s[0]
    rs = map(lambda x: int(x), s[1:])
    return (b, *rs)

In [29]:
s =rhpix_string_to_suid('N23331748')
s

('N', 2, 3, 3, 3, 1, 7, 4, 8)

In [30]:
rdggs.cell(suid=s)

In [32]:
t2 = create_rhpix_geometry(test_est["cell_id"].apply(lambda s: rdggs.cell(suid=rhpix_string_to_suid(s))))

In [33]:
t2.head(2)

,cell_id,geometry
0,N23331748,"POLYGON ((21.81076 58.22632, 21.83118 58.22632..."
1,N23331771,"POLYGON ((21.82066 58.24113, 21.84109 58.24113..."


In [4]:
import numba